# LMEM

In [ ]:
from utils.LMEM_settings import *

### Preparation R and mask for (G)LMER function

In [ ]:
%load_ext rpy2.ipython
rpy2.rinterface_lib.conversion._cchar_to_str = cchar_to_str
rpy2.rinterface_lib.conversion._cchar_to_str_with_maxlen = cchar_to_str_with_maxlen
%R library(lme4)
%R library(lmerTest)
%R library(ciTools)
%R library('nlme')
%R library('itsadug')
# %R library('itsadug')

In [ ]:
def create_glmer_in_R(formula:str, data:pd.DataFrame, family:str=LMEM_FAMILY_GAUSSIAN, description:str='model', gaussian_link:str=LMEM_ERROR_IDENTITY):
    '''
    interface for the (g)lmer functions in R (lme4 / lmerTest packages)
    input given directly to the R function, then all possible statistics are returned (factors, ...)

    Arguments:
    * formula: a two-sided linear formula object describing both the fixed-effects and random-effects part of the model, with the response on the left of a ~ operator and the terms, separated by + operators, on the right. Random-effects terms are distinguished by vertical bars ("|") separating expressions for design matrices from grouping factors.
    * data: a data frame containing the variables named in formula.
    * family: a description of the error distribution and link function to be used in the model. Possible families:
        - binomial --> "logit" only
        - gaussian --> "identity", "log"
        - Gamma --> "identity" only
        - Poisson --> "identity" only
    * description: a description of the model, required for storing the model file
    * gaussian_link: the link function between model and result for gaussian
        - "identity"
        - "log"
        
    returns: a dictionary containing the model and various characteristics / statistics about it from R
    '''
    print(formula)
    assert (family == LMEM_FAMILY_GAUSSIAN and (gaussian_link==LMEM_ERROR_IDENTITY or gaussian_link==LMEM_ERROR_LOG)) or \
        family in [LMEM_FAMILY_BINOMIAL, LMEM_FAMILY_GAMMA, LMEM_FAMILY_POISSON] 
    if family == LMEM_FAMILY_GAUSSIAN and gaussian_link==LMEM_ERROR_IDENTITY:
        %R -i formula -i data -o m m <- lmer(formula, data = data)
        %R -i m -o aic aic <- AIC(m)
        %R -i m -o bic bic <- BIC(m)
    else:
        # set together family and link function
        if family == LMEM_FAMILY_GAUSSIAN and gaussian_link==LMEM_ERROR_LOG:
            %R fam <- gaussian(link="log")
        elif family == LMEM_FAMILY_BINOMIAL:
            %R fam <- binomial()
        elif family == LMEM_FAMILY_GAMMA:
            %R fam <- Gamma("identity")
        elif family == LMEM_FAMILY_POISSON:
            %R fam <- poisson("identity")
        %R -i formula -i data -o m m <- glmer(formula, data = data, family = fam)
    %R -o m_summary m_summary <- summary(m)
    %R print(m_summary)
    %R -o m_anova m_anova <- anova(m)
    %R print(m_anova)
    %R -o m_fixedeffects m_fixedeffects <- fixef(m)
        # %R -i data -o m_confidence m_confidence <- add_ci(data, m, nSims = 300)
    %R -o m_terms m_terms <- terms(m)
    %R -o m_randomeffects m_randomeffects <- ranef(m)
    %R -o m_sum_coef m_sum_coef <- coef(summary(m))
    model = {LMEM_RESULT_MODEL:m, 
             LMEM_RESULT_SUMMARY:m_summary, 
             LMEM_RESULT_ANOVA: m_anova, 
             LMEM_RESULT_FIXED_EFFECTS: m_fixedeffects, 
             LMEM_RESULT_TERMS: m_terms,
             LMEM_RESULT_RANDOM_EFFECTS: m_randomeffects, 
             LMEM_RESULT_COEFFICIENTS: m_sum_coef,
             LMEM_RESULT_FORMULA: formula, 
             LMEM_RESULT_FAMILY: family
    }
    if family == LMEM_FAMILY_GAUSSIAN and gaussian_link==LMEM_ERROR_IDENTITY:
        model['AIC'] = aic
        model['BIC'] = bic
        print('AIC', aic, 'BIC', bic)
    #     %R -o m_confidence m_confidence <- confint(m)
    #     %R print(m_confidence)
    #     model[LMEM_RESULT_CONF_INT] = m_confidence
    
    with open(get_LMEM_model_path(description, model=True),'w') as f:
        json.dump(model, f, indent=4, cls=RTypesEncoder)
    return model